In [ ]:
import pandas as pd 
import numpy as np
import requests
!conda install -c conda-forge geopy --yes 
import json
from geopy.geocoders import Nominatim
import geocoder
from bs4 import BeautifulSoup
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium

Solving environment: / 

In [ ]:
data = requests.get("https://en.wikipedia.org/wiki/Category:Suburbs_of_Nelson,_New_Zealand").text

In [ ]:
soup = BeautifulSoup(data, 'html.parser')

In [ ]:
List = []

In [ ]:
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    List.append(row.text)

In [ ]:
df = pd.DataFrame({"Neighborhood": List})
df.head()

In [ ]:
def get_latlng(neighborhood):
    coords = None
    while(coords is None):
        g = geocoder.arcgis('{}, Nelson, Newzealand'.format(neighborhood))
        coords = g.latlng
    return coords

In [ ]:
cords = [ get_latlng(neighborhood) for neighborhood in df["Neighborhood"].tolist() ]

In [ ]:
df1 = pd.DataFrame(cords, columns=['Latitude', 'Longitude'])

In [ ]:
df['Latitude'] = df1['Latitude']
df['Longitude'] = df1['Longitude']

In [ ]:
df.to_csv("df.csv", index=False)

In [ ]:
address = 'Nelson,Newzealand'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Nelson,Newzealand {}, {}.'.format(latitude, longitude))

In [ ]:
map_Nl = folium.Map(location=[latitude, longitude], zoom_start=12)

In [ ]:
for lat, lng, neighborhood in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.5).add_to(map_Nl)  
    
map_Nl

In [ ]:
CLIENT_ID = 'AXBUNJUE3C4OQAUPPEFYQ2LZYBZEMFFTEVHUM0XLCEU4TO3J' 
CLIENT_SECRET = '2NPC42A4ARUSFMZE4ZDAJZR3GXLDJBYKDAAUKLPOFQO04DAZ' 
VERSION = '20180605' 


In [ ]:
radius = 1000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [ ]:
venuesdf = pd.DataFrame(venues)

In [ ]:
venuesdf.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

In [ ]:
venues_df.groupby(["Neighborhood"]).count()


In [ ]:
Nl_onehot = pd.get_dummies(venuesdf[['VenueCategory']], prefix="", prefix_sep="")

In [ ]:
Nl_onehot['Neighborhoods'] = venuesdf['Neighborhood'] 

In [ ]:
fixed_columns = [Nl_onehot.columns[-1]] + list(Nl_onehot.columns[:-1])
Nl_onehot = Nl_onehot[fixed_columns]


In [ ]:
grouped = Nl_onehot.groupby(["Neighborhoods"]).mean().reset_index()


In [ ]:
malls = grouped[["Neighborhoods","Shopping Mall"]]

In [ ]:
kclusters = 3

clustering = malls.drop(["Neighborhoods"], 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(clustering)

kmeans.labels_[0:10]

In [ ]:
kl_merged = kl_mall.copy()


In [ ]:
kl_merged["Cluster Labels"] = kmeans.labels_

In [ ]:
kl_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)


In [ ]:
kl_merged = kl_merged.join(kl_df.set_index("Neighborhood"), on="Neighborhood")

In [ ]:
kl_merged.sort_values(["Cluster Labels"], inplace=True)
kl_merged

In [ ]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
markers_colors = []
for lat, lon, poi, cluster in zip(kl_merged['Latitude'], kl_merged['Longitude'], kl_merged['Neighborhood'], kl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [ ]:
map_clusters.save('map_clusters.html')